In [1]:
import pandas as pd
import numpy as np

In [2]:
d=pd.read_csv("Heart Attack Data Set.csv")
d.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
nv=d.isnull().sum()
nv

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [4]:
d.shape

(303, 14)

In [5]:
import seaborn as sns
x=d.iloc[:,0:-1]
y=d.iloc[:,-1]

In [6]:
x

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


In [7]:
y

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64

In [8]:
from sklearn.feature_selection import SelectKBest,f_regression
selector=SelectKBest(f_regression,k='all').fit(x,y)
a=selector.scores_


In [9]:
sf=[i for i,score in zip(x.columns,a) if score<10]

In [10]:
sf
sf1=x.drop(columns=sf)

In [11]:
sf1

,age,sex,cp,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,150,0,2.3,0,0,1
1,37,1,2,187,0,3.5,0,0,2
2,41,0,1,172,0,1.4,2,0,2
3,56,1,1,178,0,0.8,2,0,2
4,57,0,0,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...
298,57,0,0,123,1,0.2,1,0,3
299,45,1,3,132,0,1.2,1,0,3
300,68,1,0,141,0,3.4,1,2,3
301,57,1,0,115,1,1.2,1,1,3


In [12]:
x
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)
sf1=sc.fit_transform(sf1)

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)
sf_1,sf_2,y_1,y_2=train_test_split(sf1,y,test_size=0.3,random_state=1)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
m1=RandomForestClassifier(min_impurity_decrease=0.01)
m1.fit(X_train,y_train)

y1=m1.predict(X_test)
y2=accuracy_score(y1,y_test)
y2


0.7912087912087912

In [15]:
m2=RandomForestClassifier(min_impurity_decrease=0.01)
m2.fit(sf_1,y_1)
y3=m2.predict(sf_2)
y4=accuracy_score(y3,y_2)
y4
m3=RandomForestClassifier()


In [16]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
m4=SVC()
m5=LogisticRegression()
estimators=[('m1',m3),('m2',m4),('m3',m5)]

            

In [17]:
from sklearn.model_selection import cross_val_score
for e in estimators:
    z=cross_val_score(e[1],sf_1,y_train,cv=10,scoring='accuracy')
    print(np.round(np.mean(z),2))



0.86
0.87
0.83


In [18]:
m6=VotingClassifier(estimators=estimators)
z1=cross_val_score(m6,sf_1,y_train,cv=10,scoring='accuracy')
np.round(np.mean(z1),2)

0.87

In [19]:
m5.fit(sf_1,y_train)
y6=m5.predict(sf_2)
y7=accuracy_score(y6,y_test)
y7

0.8131868131868132

In [24]:
from sklearn.neighbors import KNeighborsClassifier
a=[]
for i in range(1,20):
    clf=KNeighborsClassifier(n_neighbors=i)
    clf.fit(sf_1,y_train)
    k1=clf.predict(sf_2)
    k2=accuracy_score(k1,y_test)
    a.append(k2)

In [25]:
a

[0.7142857142857143,
 0.6593406593406593,
 0.7252747252747253,
 0.7582417582417582,
 0.7802197802197802,
 0.7692307692307693,
 0.7912087912087912,
 0.7802197802197802,
 0.8021978021978022,
 0.8241758241758241,
 0.8351648351648352,
 0.8241758241758241,
 0.8461538461538461,
 0.8351648351648352,
 0.8461538461538461,
 0.8351648351648352,
 0.8461538461538461,
 0.8461538461538461,
 0.8461538461538461]

In [26]:
clf=KNeighborsClassifier(n_neighbors=13)
clf.fit(sf_1,y_train)
k1=clf.predict(sf_2)
k2=accuracy_score(k1,y_test)
k2

0.8461538461538461

In [27]:
import joblib
joblib.dump(clf,'knn_model.pkl')

['knn_model.pkl']